<a href="https://colab.research.google.com/github/ibr14060/MultiTask-FIFO-Queue-using-Ray/blob/main/Distributed%20Web-based%20System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install the ray package
!pip install ray

In [ ]:
# define first function
def factorial(x):
  fac=1
  for i in range (1,x+1):
    fac= fac * i
  return(fac)

print(factorial(5))

120


In [ ]:
#define the second function
def add(x,y):
  return (x+y)

print(add(2,3))

5


In [ ]:
# define the third function
def square(z):
  return(z*z)

print(square(5))

25


In [ ]:
#define the fourth function

def get(w):
  return(w)

print(get(7))

7


In [ ]:
#defint the fifth function

def addten(a):
  return(a+10)

print(addten(13))

23


In [ ]:
# create a class for queue and add function enqueue for adding functions into the queue and function dequeue to remove from the queue
class Queue:
    def __init__(self):
        self.queue = []

    def enqueue(self, name, arguments, attributes):
        entry = (name, arguments, attributes)
        self.queue.append(entry)

    def dequeue(self):
        if len(self.queue) == 0:
            return None
        return self.queue.pop(0)


In [ ]:
import ray
import time

# Initialize Ray
ray.init()

@ray.remote
def execute_job(job):
    function_name, arguments, attributes = job  # Unpack the job come to get each tuble alone
    time.sleep(1)  # Adjust the sleep duration as needed
    result = function_name(*arguments) # get the result of each method
    return result # return the result

def dequeue_then_execute(queue):
    while True:
        # Dequeue two functions at a time
        first_job = queue.dequeue()
        second_job = queue.dequeue()

        # check if there are any jobs left if so exit the while loop
        if first_job is None and second_job is None:
            break

        # check if there are two jobs or only one job
        if first_job is not None and second_job is not None:

            function_name1, arguments1, attributes1 = first_job
            function_name2, arguments2, attributes2 = second_job

            # Check if the two functions have the same attributes or not
            overlap = set(attributes1).intersection(set(attributes2))

            if overlap:
                # If there's overlap, execute the first one then the second
                print("Executing first function with name", function_name1.__name__, " followed by second one", function_name2.__name__)
                result1 = ray.get(execute_job.remote(first_job))
                print("Executed", function_name1.__name__, "with arguments:", arguments1, "Result:", result1)
                result2 = ray.get(execute_job.remote(second_job))
                print("Executed", function_name2.__name__, "with arguments:", arguments2, "Result:", result2)
            else:
                # If no overlap, execute both at the same time
                print("Executing first function with name", function_name1.__name__, "and second one ", function_name2.__name__, "concurrently")
                result1, result2 = ray.get([execute_job.remote(first_job), execute_job.remote(second_job)])
                print("Executed", function_name1.__name__, "with arguments:", arguments1, "Result:", result1)
                print("Executed", function_name2.__name__, "with arguments:", arguments2, "Result:", result2)
        # if there is one remaining job run it alone
        if first_job is not None and second_job is None:
            function_name1, arguments1, attributes1 = first_job
            print("Executing", function_name1.__name__, "and there are no other jobs")
            result1 = ray.get(execute_job.remote(first_job))
            print("Executed", function_name1.__name__, "with arguments:", arguments1, "Result:", result1)



2024-05-13 17:29:21,070	INFO worker.py:1749 -- Started a local Ray instance.


In [ ]:

job_queue = Queue() # initialize an empty queue

# Enqueue jobs into the queue
job_queue.enqueue(add, (6,3), ['h','y']) # enqueue the add function with attributes h and y
job_queue.enqueue(factorial, (5,), ['x']) # enqueue the factorial function with x
job_queue.enqueue(addten, (5,), ['y']) # enqueue addten function with y
job_queue.enqueue(square, (5,), ['l']) # enqueue square function with l
job_queue.enqueue(get, (5,), ['m']) # enqueue get function with m

# Call dequeue_and_execute function
dequeue_then_execute(job_queue)

# Shut down Ray
ray.shutdown()

Executing first function with name add and second one  factorial concurrently
Executed add with arguments: (6, 3) Result: 9
Executed factorial with arguments: (5,) Result: 120
Executing first function with name addten and second one  square concurrently
Executed addten with arguments: (5,) Result: 15
Executed square with arguments: (5,) Result: 25
Executing get and there are no other jobs
Executed get with arguments: (5,) Result: 5


In [ ]:

job_queue = Queue() # initialize an empty queue

# Enqueue jobs into the queue
job_queue.enqueue(add, (6,3), ['x','y']) # enqueue the add function with attributes x and y
job_queue.enqueue(factorial, (5,), ['x']) # enqueue the factorial function with x
job_queue.enqueue(addten, (5,), ['y']) # enqueue addten function with y
job_queue.enqueue(square, (5,), ['l']) # enqueue square function with l
job_queue.enqueue(get, (5,), ['m']) # enqueue get function with m

# Call dequeue_and_execute function
dequeue_then_execute(job_queue)

# Shut down Ray
ray.shutdown()

Executing first function with name add  followed by second one factorial
Executed add with arguments: (6, 3) Result: 9
Executed factorial with arguments: (5,) Result: 120
Executing first function with name addten and second one  square concurrently
Executed addten with arguments: (5,) Result: 15
Executed square with arguments: (5,) Result: 25
Executing get and there are no other jobs
Executed get with arguments: (5,) Result: 5
